# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,17.56,93,100,3.35,SH,1712857067
1,1,blythe,33.6103,-114.5964,27.25,16,0,3.09,US,1712857067
2,2,tiksi,71.6872,128.8694,-28.37,96,24,6.34,RU,1712857068
3,3,bonthe,7.5264,-12.5050,28.74,67,92,4.56,SL,1712856898
4,4,port-aux-francais,-49.3500,70.2167,6.14,96,98,6.48,TF,1712857069


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_city_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.3,
    color = "City"
)

# Display the map
map_city_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp']<=21) & (city_data_df['Max Temp']>=0)\
                            & (city_data_df['Humidity']<70) & (city_data_df['Humidity']>=10),:]

# Drop any rows with null values
ideal_city_df.dropna(how="any")

# Display sample data
ideal_city_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,ushuaia,-54.8000,-68.3000,8.79,61,0,11.83,AR,1712857060
11,11,pingliang,35.5392,106.6861,13.06,65,0,1.78,CN,1712857074
12,12,shihezi,44.3000,86.0333,14.58,48,52,2.64,CN,1712857075
15,15,puerto natales,-51.7236,-72.4875,10.25,57,0,2.06,CL,1712857077
21,21,college,64.8569,-147.8028,2.32,66,100,3.09,US,1712856852
34,34,heber city,40.5069,-111.4132,15.90,33,3,0.45,US,1712857093
38,38,eldorado at santa fe,35.5264,-105.9347,14.79,33,0,6.69,US,1712857096
47,47,pontassieve,43.7737,11.4282,20.63,57,0,2.57,IT,1712857102
51,51,mount pearl,47.5166,-52.7813,5.18,69,20,5.14,CA,1712856839
66,66,moires,35.0532,24.8744,17.42,61,62,2.23,GR,1712857116


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
8,ushuaia,AR,-54.8000,-68.3000,61,
11,pingliang,CN,35.5392,106.6861,65,
12,shihezi,CN,44.3000,86.0333,48,
15,puerto natales,CL,-51.7236,-72.4875,57,
21,college,US,64.8569,-147.8028,66,
34,heber city,US,40.5069,-111.4132,33,
38,eldorado at santa fe,US,35.5264,-105.9347,33,
47,pontassieve,IT,43.7737,11.4282,57,
51,mount pearl,CA,47.5166,-52.7813,69,
66,moires,GR,35.0532,24.8744,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":'accommodation.hotel',
    "filter":'',
    "limit":5,
    "bias":'',
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    long = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
pingliang - nearest hotel: 平凉新都酒店
shihezi - nearest hotel: 石河子宾馆
puerto natales - nearest hotel: Dorotea Patagonia Hostel
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
heber city - nearest hotel: Swiss Alps Inn
eldorado at santa fe - nearest hotel: No hotel found
pontassieve - nearest hotel: Hotel Moderno
mount pearl - nearest hotel: Sandman Signature St. John's Hotel
moires - nearest hotel: Hotel Olympic
ancud - nearest hotel: Cabañas
alexandria - nearest hotel: Smoha Zahran Haus (7th floor)
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
valdivia - nearest hotel: Esmeralda
joplin - nearest hotel: Homewood Suites by Hilton Joplin, MO
isafjordur - nearest hotel: Hótel Horn
grand falls-windsor - nearest hotel: Hotel Robin Hood
alice springs - nearest hotel: Aurora Alice Springs
ushtobe - nearest hotel: No hotel found
luderitz - nearest hotel: Seaview Hotel Zum Sperrgeb

,City,Country,Lat,Lng,Humidity,Hotel Name
8,ushuaia,AR,-54.8000,-68.3000,61,Apart Hotel Aires del Beagle
11,pingliang,CN,35.5392,106.6861,65,平凉新都酒店
12,shihezi,CN,44.3000,86.0333,48,石河子宾馆
15,puerto natales,CL,-51.7236,-72.4875,57,Dorotea Patagonia Hostel
21,college,US,64.8569,-147.8028,66,BEST WESTERN PLUS Pioneer Park Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.3,
    color = "City",
    hover_cols=['Hotel Name','Country']
)

# Display the map
map_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)